In [ ]:
import os
import sys
import json

import requests

import time

import pandas as pd

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df = pd.read_csv("../data/events.csv", index_col=0)
len(df)
df.head(3)

In [ ]:
for row in df.iloc[:550].iterrows():
    d = row[1].fillna('nan') #jsondecode does not allow NaN values
    d = dict(d)
    response = requests.post(
        'http://localhost:5010/ingest/', 
        data=json.dumps(d, ensure_ascii=True, skipkeys=True),
        headers={"Content-type": "application/json"})
    time.sleep(.1)
    # print(response.text)
print('FINISHED')

In [ ]:
from ast import literal_eval
from functools import lru_cache, wraps
from datetime import datetime, timedelta

def timed_lru_cache(seconds: int, maxsize: int = 128):
    def wrapper_cache(func):
        func = lru_cache(maxsize=maxsize)(func)
        func.lifetime = timedelta(seconds=seconds)
        func.expiration = datetime.utcnow() + func.lifetime
        @wraps(func)
        def wrapped_func(*args, **kwargs):
            if datetime.utcnow() >= func.expiration:
                func.cache_clear()
                func.expiration = datetime.utcnow() + func.lifetime
            return func(*args, **kwargs)
        return wrapped_func
    return wrapper_cache


@timed_lru_cache(seconds=5)
def get_events(event):
    return event


In [ ]:
for row in df.iloc[:500].iterrows():
    d = row[1].fillna('nan') #jsondecode does not allow NaN values
    d = dict(d)
    get_events(str(d))
